In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --pre 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-blsdgznb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-blsdgznb
  Resolved https://github.com/facebookresearch/detectron2.git to commit a1ce2f956a1d2212ad672e3c47d53405c2fe4312
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of hydra-core to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.3 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp312-cp312-linux_x86_64.whl size=6352512 sha256=a3ac7c8aec

In [ ]:
# Tree Instance Segmentation - Dataset Registration for Detectron2

import os
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

# Configuration

COCO_JSON_DIR = "/content/drive/MyDrive/mask2former_task/coco_format"
IMAGE_ROOT = "/content/drive/MyDrive/mask2former_task/preprocessed_data"

TRAIN_DATASET_NAME = "tree_train"
VAL_DATASET_NAME = "tree_val"

# Registration Function
def register_tree_dataset():
    """
    Register the tree instance segmentation dataset with Detectron2.

    This function:
    1. Registers both train and val splits
    2. Sets up metadata (class names, colors)
    3. Verifies the registration was successful
    """

    print("\n" + "="*70)
    print("REGISTERING TREE DATASET WITH DETECTRON2")
    print("="*70 + "\n")

    # Register Training Set
    train_json = os.path.join(COCO_JSON_DIR, "instances_train.json")
    train_images = os.path.join(IMAGE_ROOT, "train_PNG")

    print(f"📝 Registering training set: '{TRAIN_DATASET_NAME}'")
    print(f"   JSON: {train_json}")
    print(f"   Images: {train_images}")

    if not os.path.exists(train_json):
        raise FileNotFoundError(f"Training JSON not found: {train_json}")
    if not os.path.exists(train_images):
        raise FileNotFoundError(f"Training images directory not found: {train_images}")

    register_coco_instances(
        name=TRAIN_DATASET_NAME,
        metadata={},
        json_file=train_json,
        image_root=train_images
    )
    print("   ✓ Training set registered successfully\n")

    # Register Validation Set
    val_json = os.path.join(COCO_JSON_DIR, "instances_val.json")
    val_images = os.path.join(IMAGE_ROOT, "val_PNG")

    print(f"📝 Registering validation set: '{VAL_DATASET_NAME}'")
    print(f"   JSON: {val_json}")
    print(f"   Images: {val_images}")

    if not os.path.exists(val_json):
        raise FileNotFoundError(f"Validation JSON not found: {val_json}")
    if not os.path.exists(val_images):
        raise FileNotFoundError(f"Validation images directory not found: {val_images}")

    register_coco_instances(
        name=VAL_DATASET_NAME,
        metadata={},
        json_file=val_json,
        image_root=val_images
    )
    print("Validation set registered successfully\n")

    # Set Metadata
    print(" Setting dataset metadata...")

    metadata = {
        "thing_classes": ["tree"],
        "thing_colors": [(34, 139, 34)],
    }

    # Apply metadata to both datasets
    MetadataCatalog.get(TRAIN_DATASET_NAME).set(**metadata)
    MetadataCatalog.get(VAL_DATASET_NAME).set(**metadata)

    print("Metadata configured")
    print(f"      - Classes: {metadata['thing_classes']}")
    print(f"      - Colors: {metadata['thing_colors']}\n")

    # Verify Registration
    print("🔍 Verifying registration...")

    # Check training dataset
    train_dicts = DatasetCatalog.get(TRAIN_DATASET_NAME)
    train_metadata = MetadataCatalog.get(TRAIN_DATASET_NAME)
    print(f"   ✓ Training: {len(train_dicts)} images loaded")
    print(f"      - Classes: {train_metadata.thing_classes}")

    # Check validation dataset
    val_dicts = DatasetCatalog.get(VAL_DATASET_NAME)
    val_metadata = MetadataCatalog.get(VAL_DATASET_NAME)
    print(f"   ✓ Validation: {len(val_dicts)} images loaded")
    print(f"      - Classes: {val_metadata.thing_classes}\n")

    # Sample Data Inspection
    print("Sample data inspection...")

    sample = train_dicts[0]
    print(f"\n   Sample image: {sample['file_name']}")
    print(f"   - Image ID: {sample['image_id']}")
    print(f"   - Dimensions: {sample['height']}x{sample['width']}")
    print(f"   - Annotations: {len(sample['annotations'])} instances")

    if len(sample['annotations']) > 0:
        ann = sample['annotations'][0]
        print(f"\n   Sample annotation:")
        print(f"   - Category ID: {ann['category_id']}")
        print(f"   - Bbox: {ann['bbox']}")
        print(f"   - Area: {ann.get('area', 'N/A')}")
        print(f"   - Segmentation type: {type(ann['segmentation']).__name__}")

    print("\n" + "="*70)
    print("DATASET REGISTRATION COMPLETE")
    print("="*70)
    print(f"\nDataset names for training config:")
    print(f"   - TRAIN: '{TRAIN_DATASET_NAME}'")
    print(f"   - VAL:   '{VAL_DATASET_NAME}'")
    print("="*70 + "\n")

    return TRAIN_DATASET_NAME, VAL_DATASET_NAME


# VISUALIZATION FUNCTION

def visualize_registered_dataset(dataset_name, num_samples=3):
    """
    Visualize some samples from the registered dataset.

    """
    import cv2
    import random
    import matplotlib.pyplot as plt
    from detectron2.utils.visualizer import Visualizer

    print(f"\n{'='*70}")
    print(f"VISUALIZING DATASET: {dataset_name}")
    print(f"{'='*70}\n")

    dataset_dicts = DatasetCatalog.get(dataset_name)
    metadata = MetadataCatalog.get(dataset_name)

    samples = random.sample(dataset_dicts, min(num_samples, len(dataset_dicts)))

    fig, axes = plt.subplots(1, len(samples), figsize=(6*len(samples), 6))
    if len(samples) == 1:
        axes = [axes]

    for idx, (sample, ax) in enumerate(zip(samples, axes)):
        img = cv2.imread(sample["file_name"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        visualizer = Visualizer(img, metadata=metadata, scale=1.0)
        vis = visualizer.draw_dataset_dict(sample)

        ax.imshow(vis.get_image())
        ax.set_title(f"Image {idx+1}\n{len(sample['annotations'])} instances",
                     fontsize=12)
        ax.axis('off')

    plt.tight_layout()

    output_path = os.path.join(COCO_JSON_DIR, f"{dataset_name}_samples.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.show()

    print(f"✓ Visualization saved to: {output_path}\n")



def print_dataset_statistics(dataset_name):
    """
    Print detailed statistics about the registered dataset.

    """
    import numpy as np

    print(f"\n{'='*70}")
    print(f"DATASET STATISTICS: {dataset_name}")
    print(f"{'='*70}\n")

    dataset_dicts = DatasetCatalog.get(dataset_name)
    metadata = MetadataCatalog.get(dataset_name)

    total_images = len(dataset_dicts)
    total_annotations = sum(len(d['annotations']) for d in dataset_dicts)

    instances_per_image = [len(d['annotations']) for d in dataset_dicts]

    all_areas = []
    for d in dataset_dicts:
        for ann in d['annotations']:
            all_areas.append(ann.get('area', 0))

    print(f" Overall Statistics:")
    print(f"   - Total images: {total_images}")
    print(f"   - Total instances: {total_annotations}")
    print(f"   - Classes: {metadata.thing_classes}")

    print(f"\n Instance Distribution:")
    print(f"   - Mean per image: {np.mean(instances_per_image):.2f}")
    print(f"   - Median per image: {np.median(instances_per_image):.0f}")
    print(f"   - Min per image: {np.min(instances_per_image)}")
    print(f"   - Max per image: {np.max(instances_per_image)}")

    print(f"\n Instance Area (pixels):")
    print(f"   - Mean: {np.mean(all_areas):.2f}")
    print(f"   - Median: {np.median(all_areas):.2f}")
    print(f"   - Min: {np.min(all_areas):.2f}")
    print(f"   - Max: {np.max(all_areas):.2f}")

    print(f"\n{'='*70}\n")


def main():
    """
    Main function to register and verify the dataset.
    """
    try:
        # Register dataset
        train_name, val_name = register_tree_dataset()

        print_dataset_statistics(train_name)
        print_dataset_statistics(val_name)

        print("Creating visualizations...")
        visualize_registered_dataset(train_name, num_samples=3)
        visualize_registered_dataset(val_name, num_samples=3)

        print("\n" + "="*70)
        print(" SUCCESS! Dataset is ready for training")
        print("="*70)
        print("\n Use these dataset names in your training config:")
        print(f"   cfg.DATASETS.TRAIN = ('{train_name}',)")
        print(f"   cfg.DATASETS.TEST = ('{val_name}',)")
        print("="*70 + "\n")

    except Exception as e:
        print(f"\n Error during registration: {str(e)}")
        raise


if __name__ == "__main__":
    main()

Output hidden; open in https://colab.research.google.com to view.